## Purpose
Tests during development of vvmtools, plottools, etc.

### Import

In [82]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import *
import glob
import logging

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn.colors.xkcd_rgb as c
import cmaps
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
import vvmtools as vvmtools_aaron

## VVMtools (data-related)

### Plan

- [ ] func: convert_to_agird()
- [ ] func: cal_tke(convert_to_agrid:bool)
- [ ] func: cal_enstrophy(convert_to_agrid:bool)
- [ ] func: cal_turb_flux(convert_to_agrid:bool)

### Development

In [5]:
testcase = '/data/mlcloud/ch995334/VVM/DATA/pbl_mod_wfire_coastal_s1/'

In [80]:
class VVMtools(vvmtools_aaron.VVMTools):
    def __init__(self, casepath):
        super().__init__(casepath)
        
        self.TIMESTEPS = len(glob.glob(f"{casepath}/archive/*Dynamic*.nc"))

    
    def convert_to_agrid(self, var, time):
        """
        Default: Interpolate the entire domain on the designated time step.
        """
        # Wind field (u, v, w)
        if var == 'u':
            u_org   = self.get_var('u', time, numpy=True)
            u_agrid = (u_org[..., 1:]+u_org[..., :-1])/2   # itp_dim: (z, y, x) -> (x)
            return u_agrid[1:, :-1, :]                     # discard the last y and first z
        elif var == 'v':
            v_org   = self.get_var('v', time, numpy=True)
            v_agrid = (v_org[:, 1:, :]+v_org[:, :-1, :])/2 # itp_dim: (z, y, x) -> (y)
            return v_agrid[1:, :, :-1]                     # discard the last x and first z
        elif var == 'w':
            w_org   = self.get_var('w', time, numpy=True)
            w_agrid = (w_org[1:, ...]+w_org[:-1, ...])/2   # itp_dim: (z, y, x) -> (x)
            return w_agrid[:, :-1, :-1]                    # discard the last x and y
        
        # Vorticity field(eta, xi, zeta)
        elif var == 'eta':
            # Check dimension for the dynamic eta
            eta_temp = self.get_var('eta', 0, (1, 1, 1, 1, 1, 1), numpy=True)
            if len(eta_temp.shape) < 3:
                eta_org = self.get_var('eta_2', time, numpy=True)
            else:
                eta_org = self.get_var('eta', time, numpy=True)
            eta_agrid   = (eta_org[1:, :, 1:] +eta_org[1:, :, :-1]+      # itp_dim: (z, y, x)
                           eta_org[:-1, :, 1:]+eta_org[:-1, :, :-1])/4.  # -> (z, x)
            return eta_agrid[:, :-1, :]                                  # discard the last y
        elif var == 'xi':
            xi_org      = self.get_var('xi', time, numpy=True)  
            xi_agrid    = (xi_org[1:, 1:, :] +xi_org[1:, :-1, :]+        # itp_dim: (z, y, x)
                           xi_org[:-1, 1:, :]+xi_org[:-1, :-1, :])/4.    # -> (z, y)
            return xi_agrid[:, :, :-1]                                   # discard the last x
        elif var == 'zeta':
            zeta_org    = self.get_var('zeta', time, numpy=True)
            zeta_agrid  = (zeta_org[:, 1:, 1:] +zeta_org[:, 1:, :-1]+    # itp_dim: (z, y, x)
                           zeta_org[:, :-1, 1:]+zeta_org[:, :-1, :-1])/4.# -> (y, x)
            return zeta_agrid[1:, :, :]                                  # discard the first z
        
        # Theta (standard)
        elif var == 'th':
            th_org      = self.get_var('th', time, numpy=True)
            return th_org[1:, :-1, :-1]       # discard the first z, last y, last x
        
    def cal_TKE(self, time, domain_range, conv_agrid:bool=True):
        """
        Calculate TKE over the designated domain range and on the specified time step.
        Default: u, v, w will be converted to a-grid before calculation.
        """
        if conv_agrid:
            if domain_range[3] > 63:
                logging.warning("")
        u = np.squeeze(self.get_var("u", t, numpy=True))
        v = np.squeeze(self.get_var("v", t, numpy=True))
        w = np.squeeze(self.get_var("w", t, numpy=True))
        # POSSIBLE TODO: argument xarray:bool 
        # -> might be more convenient to entail variable info
        
        return np.nanmean((u**2+v**2+w**2)/2, axis=(1, 2))

# Build test instance
test_vvmtools = VVMtools(testcase)

### Testing

In [81]:
print(test_vvmtools.convert_to_agrid('th', time=100).shape)
print(test_vvmtools.convert_to_agrid('th', time=100).max())

(49, 127, 127)
308.13858


In [55]:
len((1, 1, 1))

3

In [32]:
test_vvmtools.DIM

{'xc': array([  100.,   300.,   500.,   700.,   900.,  1100.,  1300.,  1500.,
         1700.,  1900.,  2100.,  2300.,  2500.,  2700.,  2900.,  3100.,
         3300.,  3500.,  3700.,  3900.,  4100.,  4300.,  4500.,  4700.,
         4900.,  5100.,  5300.,  5500.,  5700.,  5900.,  6100.,  6300.,
         6500.,  6700.,  6900.,  7100.,  7300.,  7500.,  7700.,  7900.,
         8100.,  8300.,  8500.,  8700.,  8900.,  9100.,  9300.,  9500.,
         9700.,  9900., 10100., 10300., 10500., 10700., 10900., 11100.,
        11300., 11500., 11700., 11900., 12100., 12300., 12500., 12700.,
        12900., 13100., 13300., 13500., 13700., 13900., 14100., 14300.,
        14500., 14700., 14900., 15100., 15300., 15500., 15700., 15900.,
        16100., 16300., 16500., 16700., 16900., 17100., 17300., 17500.,
        17700., 17900., 18100., 18300., 18500., 18700., 18900., 19100.,
        19300., 19500., 19700., 19900., 20100., 20300., 20500., 20700.,
        20900., 21100., 21300., 21500., 21700., 21900., 22

In [63]:
ds_test = xr.open_dataset(f"{testcase}/archive/pbl_mod_wfire_coastal_s1.L.Thermodynamic-000000.nc")

In [64]:
ds_test.th

<xarray.DataArray 'th' (time: 1, lev: 50, lat: 128, lon: 128)>
[819200 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1900-01-01
  * lev      (lev) float64 0.0 0.02 0.06 0.1 0.14 ... 1.78 1.82 1.86 1.9 1.94
  * lat      (lat) float64 -0.1142 -0.1124 -0.1106 ... 0.1106 0.1124 0.1142
  * lon      (lon) float64 -0.1142 -0.1124 -0.1106 ... 0.1106 0.1124 0.1142
Attributes:
    standard_name:  potential_temperature
    long_name:      potential temperature
    units:          K

In [65]:
ds_test

<xarray.Dataset>
Dimensions:  (time: 1, lev: 50, lat: 128, lon: 128)
Coordinates:
  * time     (time) datetime64[ns] 1900-01-01
  * lev      (lev) float64 0.0 0.02 0.06 0.1 0.14 ... 1.78 1.82 1.86 1.9 1.94
  * lat      (lat) float64 -0.1142 -0.1124 -0.1106 ... 0.1106 0.1124 0.1142
  * lon      (lon) float64 -0.1142 -0.1124 -0.1106 ... 0.1106 0.1124 0.1142
Data variables: (12/13)
    zc       (lev) float64 ...
    yc       (lat) float64 ...
    xc       (lon) float64 ...
    th       (time, lev, lat, lon) float32 ...
    qv       (time, lev, lat, lon) float32 ...
    qc       (time, lev, lat, lon) float32 ...
    ...       ...
    qi       (time, lev, lat, lon) float32 ...
    nc       (time, lev, lat, lon) float32 ...
    nr       (time, lev, lat, lon) float32 ...
    ni       (time, lev, lat, lon) float32 ...
    qrim     (time, lev, lat, lon) float32 ...
    brim     (time, lev, lat, lon) float32 ...
Attributes:
    model_tag:     CSU VVM
    references:    http://kiwi.atmos.colostate.edu/pubs/joon-hee-tech_report.pdf
    contact:       jung@atmos.colostate.edu
    institution:   Colorado State University
    VVM_casename:  vvm_exp                                                   ...

In [69]:
np.arange(100, 25500.1, 200).shape

(128,)

## Plottools